In [1]:
# 第七节
# models.py
from django.db impor models

# Create your models here.
# tag(标签)
class Tag(models.Model):
    # 主键 自动维护生成
    name = models.CharField(max_length=30, verbose_name='标签名称')
    
    class Meta:
        # 方便在admin里看到标签的中文名
        verbose_name = '标签'
        verbose_name_plural = verbose_name
        
    # model最好每个都有，返回自身的属性
    class __unicode__(self):
        return str(self.id) # 返回字符串类型
    
# 文章分类
calss Catagory(models.Model):
    name = models.CharField(max_length=30, verbose_name='分类名称')
    index = models.INtegerField(verose_name='分类的排序')
    class Meta:
        # 方便在admin里看到标签的中文名
        verbose_name = '分类'
        verbose_name_plural = verbose_name
        
    # model最好每个都有，返回自身的属性
    class __unicode__(self):
        return str(self.id) # 返回字符串类型

SyntaxError: invalid syntax (<ipython-input-1-a730f9fb44a9>, line 2)

In [2]:
# 第七节
# setting.py
DATABASES = {
    'defalut' : {
        'ENGINE' : 'django.db.backends.mysql' # 后端
        'NAME' : 'blogdb',
        'USER': 'root',
        'PASSWORD' : '1234',
        'HOST': '', # 默认值
        'PORT' : '',
    }
}

# 自定义用户model
AUTH_USER_MODEL = 'blog.User'

# MYSQL命令行
show databases
create database blogdb
# 也可以通过管理工具创建
# 编码， 排序规则
# -*- coding: utf-8 -*-
makemigrations blog（项目名字） # 编译 生成脚本
syncdb # 第一次初始化数据库，添加一个默认管理员的账号，密码会经过加密处理

# 每次数据库有改变时，使用makemigrations [blog]加入到脚本里
# 然后使用 migrate [blog] 将更改同步到数据库

SyntaxError: invalid syntax (<ipython-input-2-2ad16cbad56e>, line 6)

In [3]:
# 第八章 admin的配置
# django.contrib:admin,auth,sessions,conmments等模块
# 1 在INSTALLED_APPSzhong总天天加django.contrib.admin
# 2 配置urls.py:url(r'~admin/', include(admin.site.urls))
# 3 在admin注册的Model，（默认方式是和自定义方式），关于admin的配置：参考文档

# setting.py
INSTALLED_APP
# urls。py
urlpatterns = [
    url(r'^admin/', include(admin.site.urls))
]
# 启动项目后，localhost：8000/admin/login/

# models.py 注册models
admin.site.register(User)

# 刷新admin管理页面

# xadmin是非官方的admin管理模块，界面表现形式比较丰富

#fields/exclude
#fieldsets,某些信息显示在一些块，另一些显示在另一个块
# list_display,列表显示的内容
# list_editable
# inlines 一个model对象关联到另一个对象
# admin.py
class ArticleAdmin(admin.ModelAdmin):
    
    
#     fields = ('title', 'desc', 'content',) # 只展示title，desc，content
    list_display = ('title', 'desc', 'click_count',)
    list_display_links = ('title', 'desc',)
    list_editable = ('click_count',) # 不能同时是links和editable
    fieldsets = (
        (None, {
                'fields' : ('title', 'desc', 'content',)
        }),
        ('高级设置', {
                'classes' : ('collapse',) # 默认隐藏
                'fields' : ('click_count', 'is_recommend',)
        })
        
    )
    
admin.site.register(Article, ArtticleAdmin)

# admindocs的使用：生成自己的开发文档

SyntaxError: invalid syntax (<ipython-input-3-9887b76fe758>, line 41)

In [4]:
# 第九节 增加富文本编辑器：ckeditor/ueditor/kindeditor/tinymce/..
# django-admin中添加富文本编辑器的几种方式：
# 1 第三方库 django-ckeditor（下载然后setting中配置）
# 2 在admin中定义富文本编辑器widget
# 3 同过定义ModelAdmin的媒体文件
    # 下载kindeditor，拷到static的js中 
    # 2定义ModelAdmin的媒体文件
# admin.py
class Media:
    js = (
        '/static/js/kindeditor-4.1.10/kindeditor-min.js',
        '/static/js/kindeditor-4.1.10/lang/zh_CN.js',
        '/static/js/kindeditor-4.1.10/config.js',
    )
# 在js目录下新建config.js
# config.js
KindEditor.ready(function(k)) {
    K.create('textarea[name=content]', {
            Width:500,
            height:200, 
        })
}
# 查看页面及其源代码

SyntaxError: invalid syntax (<ipython-input-4-2b0c31f3cfd2>, line 17)

In [5]:
# 第十节 如何上传文件
# 1 在setting.py中配置MEDIA_URL和MEDIA_ROOT
# 2 urls.py 中配置路由
# 3 在model中设置图片的上传位置和路径
# 根目录新建uploads文件夹专门存上传文件
# setting.py 配置
MEDIA_URL = '/uploads/'

MEDIA_ROOT =os.path.join(BASE_DIR, 'uploads')

# url.py 配置路由
from django.conf import settings

urlpatterns = [
    url(r'^uploads/(?P<path>.*)$', \
       'django.view.static.serve', \
       {'document_root': setting.MEDIA_ROOT,}),
    url(r'^admin/', include(admin.sit.urls))
]

# models.py
class User(AbstractUser):
    avatar = models.ImageField(upload_to='avatar/%Y/%m', default='avatar/default.png'[,..])
    
# blog/migrations/upload.py
# -*- coding: utf-8 -*-
from django.http import HttpResponse
from django.conf import settings
from django.views.decorators.csrf import csrf_exempt
import os
import uuid
import json
import datetime as dt

@csrf_exempt # 装饰器，请求不再验证csrf
def upload_image(request, dir_name):
    ################## 这是编辑器所需要的格式
    #  kindeditor图片上传返回数据格式说明：
    # {"error": 1, "message": "出错信息"}
    # {"error": 0, "url": "图片地址"}
    ##################
    result = {"error": 1, "message": "上传出错"}
    files = request.FILES.get("imgFile", None) # 上传文件空间的name
    if files:
        result =image_upload(files, dir_name)
    return HttpResponse(json.dumps(result), content_type="application/json")

#目录创建
def upload_generation_dir(dir_name):
    today = dt.datetime.today()
    dir_name = dir_name + '/%d/%d/' %(today.year,today.month)
    if not os.path.exists(settings.MEDIA_ROOT + dir_name):
        os.makedirs(settings.MEDIA_ROOT + dir_name)
    return dir_name

# 图片上传
def image_upload(files, dir_name):
    #允许上传文件类型
    allow_suffix =['jpg', 'png', 'jpeg', 'gif', 'bmp']
    file_suffix = files.name.split(".")[-1]
    if file_suffix not in allow_suffix:
        return {"error": 1, "message": "图片格式不正确"}
    relative_path_file = upload_generation_dir(dir_name)
    path=os.path.join(settings.MEDIA_ROOT, relative_path_file)
    if not os.path.exists(path): #如果目录不存在创建目录
        os.makedirs(path)
    file_name=str(uuid.uuid1())+"."+file_suffix # 保证唯一性
    path_file=os.path.join(path, file_name)
    file_url = settings.MEDIA_URL + relative_path_file + file_name
    open(path_file, 'wb').write(files.file.read()) # 保存图片
    return {"error": 0, "url": file_url}

#blog_project/urls.py 定义路由
from blog.uploadd import upload_image
urlpatterns = [
    url(r'admin/upload/(?P<dir_name>[^/]+)$', upload_image, name='upload_image')
]

# kindeditor/config.js 配置config.js
KindEditor.ready(function(K){
        K.create('textarea[name=cont ent]', [
                width:'800px',
                heigght:'200px',
                uploadJson:'/admin/upload/kindeditor',
            ])
    })

SyntaxError: invalid syntax (<ipython-input-5-dfb5c9dd1a83>, line 23)

In [6]:
# 第十一节 模板的规划和设计
# Django模板中url的使用http://blog.csdn.net/huochen1994/article/details/52634686
# 如何拆分膜拜你，抽离出base模板
# 如何规划和设计模板中的block,
{% block left_content %}{endblock}
{% block custom_css %}{endblock}

# 继承
{% extends 'base.html' %}
{% load staticfiles%}
{% block left_content %}
# 填入内容

{% endblock %}

#如何使用include
{% include 'ad.html' %} # 引入一个内容ad.html是否应该自己能成页面（包含必须得js等

# 导航数据的获取
return render
{% for % }
{% endfor %}
# models.py
class Category(models.Model):
    
    class Meta:
        verbose_name = '分类'
        verbose_name_pplural = verbose_name
        ordering = ['index', 'id '] # 升序，倒序只需在前加减号’-index‘
# views.py
try:
    ctaegory_list = Category.objects.all()[:1]
except Excetion as e:
    logger.error(e)
return render(request, 'index.html', {'category_list': category_list})
# 可以分析执行时打印的sql代码

SyntaxError: invalid syntax (<ipython-input-6-7855bac57bfe>, line 4)

In [7]:
# 第十二章 分页器Paginator的使用
# 功能点：1、广告数据获取，2、最新文章数据获取（分页）
# 技术点
# 1、查询的基本操作，all()
# a、QuerySet[:1],这种方式是查询所有的结果再取其中一条数据还是只从数据库中取一条
# b、QuerySet查询时惰性的（当我们去执行all，filter，get，是不会去执行sql的，当
#  我们去查询查询结果集时才调用）--print type(name)
# 2/分页器Paginator的使用
# 3、locals()函数将当前作用域所有页面变量全部传递

# views.py
# 导入分页器及异常
def index(request):
    try:
        # 分类信息获取（导航数据）
        category_list = Category.objects.all()[:1]
        # 广告数据
        # 最新文章数据
        article_list = Article.object.all() # 在meta中定义顺序
        paginator = Paginator(article_list, 10)
        try:
            page = int(request.GET.get('page', 1))
            article_list = paginator.page(page)
        exception (EmptyPage, InvalidPage, PageNotAnInteger):# 用户胡乱输入 
            article_list = paginator.page(1)
    except Exception as e:
        logger.error(e)
    return render(request, 'index.html', locals())
# index.html
{% for article in article_list %}
{{article.title}}
{% for tag in article.tag.all %}
{{tag.name}}&nbsp #加入空格
{% endfor %}
{{ article.comment_set.all.count}}# Comment的外键是Article的主键 一对多加_set
{{article.data_publish | data:'Y-m-d'}} # 过滤器不加%
{% endfor %}

SyntaxError: invalid syntax (<ipython-input-7-c1b394592a25>, line 24)

In [8]:
# 第十三章 自定义Manager管理器
# 功能点： 文章归档
# 技术点：
# 1、使用filter（）
# 2、 values(), distinct()
# 3、django中直接使用sql的两种方式
#  SELECT DISTINCT DATE_FORMAT(data_publish, '%Y-%m') as col_date FROM blog_article ORDER
# 3.1、 raw（异常，Raw query must iniclude the primary key, 返回结果必须包含主键）
# 3.2、 excute
# 4、 尝试用优雅的方式解决一些数据处理上的问题，自定义Manager管理器
# views.py
# 文章归档
# 1、先要去获取到文章中有的年份-月份
# Article.objects.values('data_publish').distinct() 去重
# archive_list = Article.objects.raw('select distinct date_format(date_publish, '%%Y-%%m') as col_date..')')# 占位符与转义
    cursor = connection.cursor()
    cursor.execute(sql)
    row = cursor.fetchall()
    print row
# 不推荐，如果用的话数据库迁移会有麻烦

IndentationError: unexpected indent (<ipython-input-8-f851733011d8>, line 16)

In [9]:
# 第14章 自定义Manager管理器
# 自定义一个文章Model的管理器
# 1、新加一个数据处理的方法
# 2、改变原有的queryset
# 定义管理器
class ArticleManager(models.Manager):
    def distinct_date(self):
        distinct_date_list = []
        date_list =self.values('data_publish')
        for date in date_list:
            date['date_publish'].strftime('%Y%m文档存档')
            if date not in distinct_date_list:
                distinct_date_list.append(date)
        return distinct_date_list
# 关联管理器
class Article(models.Model):
    
    objects = ArticleManager()
    
# views.py
for a in Article.objects.objects.distinct_date():
    print a

# 链接 views.py
def arcchive(request):
    try:
        # 导航数据和归档信息也需要进来
        # 先获取客户端提交的信息
        year = request.GET.get('year', None)
        month = request.GET.get('month', None)
#         article_list = Article.object.all() # 在meta中定义顺序
        article_list = Articlle.objects.filter(date_publish_icontains=year+'-'+month)
        paginator = Paginator(article_list, 10)
        try:
            page = int(request.GET.get('page', 1))
            article_list = paginator.page(page)
        exception (EmptyPage, InvalidPage, PageNotAnInteger):# 用户胡乱输入 
            article_list = paginator.page(1)
    excet Exception as e:
        logger.error(e)
    return render(request, 'archive.html', locals())
# url设置 urls.py
url(r'^archive/$', archive, name='archive')

# 页面
{{request.GET.year}}/{{request.GET.monthh}}文章归档
{% for archive in archive_list %}
<a href='{% url'archive'%}?year={{ archive| slice:":4"}}&month={{archive | slice:"5:7"}'>{{archive}}</a>
{% endfor %}



SyntaxError: invalid syntax (<ipython-input-9-218d8035247d>, line 37)

In [ ]:
# 第十五章 重构一下代码
# 功能点：考虑一下代码的重构、文章排行、标签云、友情链接、标签列表页面
# 整个网站都会用到的代码放在global_setting函数中 views.py重构
# urls.py的重构，属于app的归入到app并在project的urls.py中加入include
# 分页代码的重构
# 整体代码的重构
# 评论排行 聚合函数
# views.py
from Django.db. import Count
comment_count_list = Comment.objects.values('article').annotate(comment_count=Count('article')).order_by('-comment_count')
article_comment_list = [Article.objects.get(pk=comment['article']) for comment in comment_count_list]
return locals()

# html中加入过滤器

In [ ]:
# 第十六章 如何自定义过滤器(filter也可以自定义tags)
# 1、DoesNotExist异常
# 2、safe过滤器的使用，date过滤器的使用
# 3、如何自定义一个过滤器
# date、safe（信息是安全的不需要过滤）
# 定义函数
# 注册到过滤器
# myfilter
from django import template
register = template.Library()

@register.filter[(name='month_to_upper')]
def month_to_upper():

# html中
{% load myfilter %}

In [ ]:
# 第十七章 评论
# 知识点 1、评论信息读取，注意细节（减少数据库读写次数）2、发表评论3、客户端验证和服务端验证4、csrf5、表单的使用
# 不加csrf时views中加入一个装饰器

In [ ]:
# 注册登录注销
# 知识点：1、django.contrib.auth（logout，login，authenticate
# 2、django.contrib.auth.hashers(make_password)django的加密方式
# 3、使用redirect跳转
# 4、如何判断用户已经登陆
# 5、。。。
# app/urls.py
url('^login', do_login, name='login')
url('^reg', do_reg, name='reg')
url('^logout', do_logout, name='logout')

# views.py
# 注册
def do_reg(request):
    try:
        if request.method = 'POST':
            reg_form = RegForm(request.POST)
            if reg_form.is_valid():
                # 注册
                user = User.objects.create(username=reg_form.cleaned_data['username'],
                                          email=reg_form.cleaned_data['email'],
                                          url=reg_from.cleaned_data['url'],
                                          password=make_password(reg_form.cleaned_data['password'],)
                user.save()
                # 登陆
                user.backend = 'django.contrib.auth.backends.ModelBackend' # 指定默认的登陆验证方式
                login(request, user)
                return redirt(request.POST.get('source_url')) # source_url跳转前的页面
            else:
                return render(request, 'failure.html', {'reason': reg_form.errors})
        else:
            reg_form = RegForm()
    except Exception as e:
        logger.error(e)
    return render(request, 'reg.html', locals())

# reg.html
#　记录网页来源（注册完成跳回）

In [ ]:
# 登陆
user = authenticate(username=username, password=password)
if user is not None:
    pass
else:
    return render(request, 'failure.html', {'reson': '登陆验证失败'})

In [ ]:
# 注销
def do_logout(request):
    try:
        logout(request)
    except Exception as e:
        logger.error(e)
    return redirect(request.META('HTTP_REFERER'))

# 是否登陆
user = request.user if request.user.is_authenticated() else None